In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
import sys,cv2
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import datetime

from Utils.utils import *
from Utils.f1_score import *
from ipywidgets import interact
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

import torch
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

import deepdish as dd

import timeit,gc

data_folder = 'D:/data/HPA/all/'
model_folder = 'D:/data/HPA/models/'
out_folder = 'D:/data/HPA/out/'
USE_SMALL_IMAGES = True
USE_ONLY_POI = True

print("Using GPU:",torch.cuda.is_available())
print("Using device ",torch.cuda.get_device_name(0))
print("Done.")

### Load test data

In [ ]:
%%time
if USE_ONLY_POI:
    filename = 'test_poi'
else:
    filename = 'test_all_channel_'
    
if USE_SMALL_IMAGES:
    d = dd.io.load(data_folder+filename+'_small.h5')
else:
    d = dd.io.load(data_folder+filename+'.h5')

    
X = d['X'].astype(np.float32) / 255.0 # torch likes float images

IDs = d['ID']
   
print("Shapes are:")
print(X.shape)
print("Done")

### Load model

In [ ]:
#Setup resnet model
net = models.resnet18(pretrained=False)
modelname = "baseline"

if USE_ONLY_POI:
    net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
else:
    net.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,bias=False)
    
if USE_SMALL_IMAGES:
    net.fc = nn.Linear(512, 28) #adapt last layer to allow larger input images
else:
    net.fc = nn.Linear(51200, 28) #adapt last layer to allow larger input images

if not USE_ONLY_POI:
    modelname = modelname + "_all"
    
if USE_SMALL_IMAGES:
    modelname = modelname + "_small"

net.load_state_dict(torch.load(model_folder+modelname+".model")) 
    
print("Done.")

### Run test set

In [ ]:
# Run validation
gc.collect() 
net.eval()

batch_size = 100
iterations_per_epoch = np.ceil(X.shape[0] / batch_size).astype(int)

time_per_epoch,runtime = 0,0
labels = []

for i in range(iterations_per_epoch):
    start = timeit.default_timer() #measure time
    start_idx = i * batch_size
    
    real_batch_size = np.minimum(batch_size,X.shape[0] - i*batch_size)
    if real_batch_size == 0: #in case X.shape[0] is divisible by batch size this is required
        real_batch_size = batch_size

    X_batch = torch.tensor(X[start_idx:start_idx+real_batch_size].transpose(0,3,1,2))

    # forward + backward + optimize
    outputs = net(X_batch)
    
    #store output, the for loop might be avoidable
    out = outputs.detach().numpy() > 0.5
    for label in out:
        labels.append(np.argwhere(label))
    
    
    #measure runtime
    stop = timeit.default_timer()
    time_per_epoch = 0.5 * time_per_epoch + 0.5 * (stop-start) * iterations_per_epoch
    runtime += (stop-start)
    #print performance metrics
    print('[iteration = (%3d/%d), time = %3ds, est. time per epoch = %5ds]'\
          %(i+1, iterations_per_epoch, runtime, time_per_epoch))
    
print("Done.")

### Create submission.csv

In [ ]:
print(len(labels))
print(len(IDs))
converted_labels = []
for i,label in enumerate(labels):
    converted_labels.append("")
    for nr in label:
        converted_labels[i] += str(nr[0]) + " "
    converted_labels[i] = (converted_labels[i])[:-1]

In [ ]:
d = {'Id': IDs, 'Predicted': converted_labels}
df = pd.DataFrame(data=d)
print(df)
df.to_csv(path_or_buf=out_folder + "submission_" + str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")) + ".csv", index=False)